In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 17 22:32:01 2018

@author: hassan.naseri
"""

import numpy as np
import pandas as pd
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute

# This is the number of bits used to address the dataset
# In our example, the shelf number of a fruit is a 4 bit unsigned integer.
NBITS = 4;

In [ ]:
fileurl = "https://raw.githubusercontent.com/HassanNaseri/quantum-computing-handson/master/fruits2.csv"
data_frame = pd.read_csv(fileurl)

def black_box_check(circuit, key, q):
    # Black box function to encode a winner entry in a dataset
    # Returns inverse (NOT) of 'value' arguments if 'key' and 'value' pair match,
    # otherwise it returns the 'value' argument unchanged.
    # Note: this function is not implemented efficiently, but its details are not importnat!
    data_dict = data_frame.set_index('name').T.to_dict('records')[0]
    try:
        winner = uint4(data_dict[key])
    except:
        raise ValueError('Invalid key! There is no ' + key)
    if winner&2 == 0:
        circuit.s(q[0])
    if winner&1 == 0:
        circuit.s(q[1])
    circuit.h(q[1])
    circuit.cx(q[0], q[1])
    circuit.h(q[1])
    if winner&2 == 0:
        circuit.s(q[0])
    if winner&1 == 0:
        circuit.s(q[1])

def reflection_about_average(circuit, q):
    # Reflection about average for amplitude amplification
    circuit.h(q[0])
    circuit.h(q[1])    
    circuit.x(q[0])
    circuit.x(q[1])
    circuit.h(q[1])
    circuit.cx(q[0], q[1])
    circuit.h(q[1])
    circuit.x(q[0])
    circuit.x(q[1])
    circuit.h(q[0])
    circuit.h(q[1])

In [ ]:
###############################################################################
# The main body code is here:

# The name of fruit to search for its shelf number: 
# Options: apple, banana, orange, strawberry
fruit_to_find = 'apple'

# Define reqisters and circuit
qreg = QuantumRegister(2)
creg = ClassicalRegister(2)
grover = QuantumCircuit(qreg, creg)

# Initialization of quantum registers
grover.h(qreg[0])
grover.h(qreg[1])

# Grover's amplitude amplification (only one iteration here)
black_box_check(grover, qreg)
reflection_about_average(grover, qreg)

# Add Measurement
grover.measure(qreg, creg)

# Execute and collect results
job_sim = execute(grover, "local_qasm_simulator")
sim_result = job_sim.result()

# Print result counts
print("simulation: ", sim_result)
print(sim_result.get_counts(grover))